# Football in Denmark: Where are we playing?

Imports and set magics:

In [ ]:
# %pip install git+https://github.com/alemartinello/dstapi #Installing the API (only need to do once)

In [ ]:
# %pip install pandas-datareader # Installing the data reader (only need to do once)

In [ ]:
# %pip install geopandas 
# Installing the geopandas reader (only need to do once)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib.ticker import FuncFormatter
import geopandas as gpd
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})
import ipywidgets as widgets
# from matplotlib_venn import venn2
from dstapi import DstApi # install with `pip install git+https://github.com/alemartinello/dstapi`

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


# Read data

Creating a dictionary for the columns names:

In [ ]:
columns_dict = {}
columns_dict['BLSTKOM'] = 'region'
columns_dict['AKTIVITET'] = 'activity'
columns_dict['KON'] = 'sex'
columns_dict['ALDER1'] = 'age'
columns_dict['TID'] = 'year'
# columns_dict['POP'] = 'population'
# columns_dict['INDHOLD'] = 'players'

#var_dict = {} # var is for variable
#var_dict['Football'] = 'football'

**Step 1:** Downloading all of the football-variables in merged_df01

In [ ]:
idrakt_api = DstApi('IDRAKT01')  #Creating the DST API which will allow us to interact with the API server
params = idrakt_api._define_base_params(language='en') #Creating a parameter dictionary with the language set to English
variables = params['variables'] # Returns a view, that we can edit
variables[1]['values'] = ['A22'] # Choosing football as the activity (The ID for football is A22 Using code from: https://alemartinello.com/2022/02/24/dstapi/)
print(variables)

In [ ]:
idrakt = idrakt_api.get_data(params=params) #Downloading the dataset

**Step 1A** Henter Befolkningsdata fra FOLK1AM

In [ ]:
folk_api = DstApi('FOLK1A')  #Creating the DST API which will allow us to interact with the API server
params = folk_api._define_base_params(language='en') #Creating a parameter dictionary with the language set to English
variables = params['variables'] # Returns a view, that we can edit
variables[2]['values'] = ['IALT']
variables[3]['values'] = ['TOT']
print(variables)

In [ ]:
folk = folk_api.get_data(params=params) #Downloading the dataset

In [ ]:
folk.head() #Looking at the dataset

# Clean data

**Cleaning population data (FOLK1A)**

In [ ]:
folk.drop(columns=['ALDER'],inplace=True)
folk.drop(columns=['CIVILSTAND'],inplace=True)
folk.head()

In [ ]:
folk_q1 = folk.loc[folk['TID'].str.contains('Q1')] # Selecting only the first quarter of the year
folk_q1['TID'] = folk_q1['TID'].str.replace('Q1', '')
folk_q1['KØN'] = folk_q1['KØN'].str.replace('Total', 'Sex, total')
folk_q1['OMRÅDE'] = folk_q1['OMRÅDE'].str.replace('Landsdel', 'Province')
folk_q1.head()

**Cleaning football data (IDRAKT01)**

**Step 2:** Only keep rows where the variable is in `Age, total` and afterwards deleting the coloumn.

In [ ]:
#Only keeps rows with age = 'Age, total' and afterwards deleting the age coloumn
idrakt = idrakt[idrakt['ALDER1'] == 'Age, total']
idrakt.drop(columns=['ALDER1'],inplace=True)
idrakt.head()

Summarizing provinces to regions:

In [ ]:
def map_regions(BLSTKOM):
    if BLSTKOM == 'All Denmark':
        return 'All Denmark'
    if BLSTKOM in ['Province Nordjylland']:
        return 'Region Nordjylland'
    elif BLSTKOM in ['Province Vestjylland', 'Province Østjylland']:
        return 'Region Midtjylland'
    elif BLSTKOM in ['Province Fyn', 'Province Sydjylland']:
        return 'Region Syddanmark'
    elif BLSTKOM in ['Province Østsjælland', 'Province Vest- og Sydsjælland']:
        return 'Region Sjælland'
    elif BLSTKOM in ['Province Bornholm', 'Province Byen København', 'Province København omegn', 'Province Nordsjælland']:
        return 'Region Hovedstaden'

# Apply the function to create the new variable 'county_a'
idrakt['county_a'] = idrakt['BLSTKOM'].apply(map_regions)
idrakt.head(100)

Only keeping regions and all of Denmark

In [ ]:
# Keeping rows where 'county' starts with "Landsdel" or is "All Denmark"
idrakt = idrakt[idrakt['BLSTKOM'].str.startswith('Region') | (idrakt['BLSTKOM'] == 'All Denmark')]
idrakt.head(100)

**Merging the two datasets:**

In [ ]:
idrakt['TID'] = idrakt['TID'].astype(str)
folk_q1['TID'] = folk_q1['TID'].astype(str)

merged_df = pd.merge(
    idrakt,
    folk_q1.rename(columns={'INDHOLD': 'POP'}),  # Rename INDHOLD to POP in folk_q1 before merging
    how='left',  # Keep all rows from idrakt
    left_on=['BLSTKOM', 'KON', 'TID'],  # Columns to match in idrakt
    right_on=['OMRÅDE', 'KØN', 'TID']  # Corresponding columns to match in folk_q1
)
merged_df.drop(['OMRÅDE', 'KØN'], axis=1, inplace=True) #Drops unnessecary columns

In [ ]:
merged_df.head()

**Step 3:** Rename coloums using colmns_dict

In [ ]:
#merged_df.rename(columns=columns_dict,inplace=True)
#merged_df.head()

idrakt.rename(columns=columns_dict,inplace=True)
idrakt.head()

**Step 5:** Sort the dataset by county, year and sex

In [ ]:
# Sorting the dataset by county then year and then total sex, but first we customly sort the order of Sex
# Define a custom sorting order
sort_order = ['Sex, total', 'Men', 'Women']

# Create a categorical type with the custom order
idrakt['sex'] = pd.Categorical(idrakt['sex'], categories=sort_order, ordered=True)

idrakt.sort_values(by=['county','year','sex'],inplace=True)
idrakt.reset_index(drop=True,inplace=True)
idrakt.head()

In [ ]:
# create a new column named 'county_B' with the a broader definition of the counties. If county is All Denmark, then county_B is All Denmark.
# county_B is split into Jylland, Sjælland and Fyn.
# Therefore I sum the values of the counties in Jylland, Sjælland and Fyn to get the values of the broader definition of the counties.
# Here Jylland is the counties: 'Province Nordjylland', 'Province Sydjylland', 'Province Vestjylland', 'Province Østjylland'
# Sjælland is the counties: 'Province Bornholm', 'Province Københavns omegn', 'Province Byen København', 'Province Nordsjælland', 'Province Østsjælland', 'Province Vest- og Sydsjælland'
# Fyn is the counties: 'Province Fyn'

# Define a function to map each county to its broader definition
def map_county_b(county):
    if county == 'All Denmark':
        return 'All Denmark'
    if county in ['Province Nordjylland', 'Province Sydjylland', 'Province Vestjylland', 'Province Østjylland']:
        return 'Jylland'
    elif county in ['Province Bornholm', 'Province Københavns omegn', 'Province Byen København', 'Province Nordsjælland', 'Province Østsjælland', 'Province Vest- og Sydsjælland']:
        return 'Sjælland'
    elif county == 'Province Fyn':
        return 'Fyn'

# Apply the function to create the new variable 'county_b'
idrakt['county_b'] = idrakt['county'].apply(map_county_b)

# Print the first few rows to verify the new variable
idrakt.head()


In [ ]:
# Creating a copy of the dataset where we only keep the rows with the value 'All Denmark' in the 'county' column
idrakt_all = idrakt[idrakt['county'] == 'All Denmark'].copy()
idrakt_all.drop(columns=['county'],inplace=True)
idrakt_all.head()

In [ ]:
# Sorting the total dataset by year
idrakt_all.sort_values(by='year',inplace=True)
idrakt_all.head()

# Analysis across genders

Here we calculate the total number of Danish football players split based on sex. 

In [ ]:
# Filtering out 'Sex, total' from the dataset
idrakt_stackedbar = idrakt_all[idrakt_all['sex'] != 'Sex, total']

# Grouping by 'year' and 'sex', then summing the values
grouped_idrakt_stackedbar = idrakt_stackedbar.groupby(['year', 'sex'])['value'].sum().unstack()

# Plotting the stacked bar chart
grouped_idrakt_stackedbar.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Yearly Values Summed by Sex (Excluding Total)')
plt.xlabel('Year')
plt.ylabel('# of Danish Football Players')
plt.legend(labels=['Women', 'Men'], title='Sex')
plt.tight_layout()
plt.show()

It is hard to see the relative development between the sexes and for total. Hence, we index the values and display here.

In [ ]:
index_values = idrakt_all[idrakt_all['year'] == 2014].set_index('sex')['value'] # Create a series with the 2014 values
indexed_idrakt_all = idrakt_all.set_index(['year', 'sex']) # Set the index
indexed_idrakt_all['indexed_value'] = 100*indexed_idrakt_all['value'] / indexed_idrakt_all['value'].unstack().loc[2014] # Calculate the indexed values

# Resetting index for plotting
indexed_idrakt_all = indexed_idrakt_all.reset_index() 

# Step 2: Plot the indexed values
plt.figure(figsize=(10, 6))
for sex, group in indexed_idrakt_all.groupby('sex'): # Loop over the groups
    plt.plot(group['year'], group['indexed_value'], marker='o', label=sex) 

plt.title('Relative Development Indexed to 2014 Values')
plt.xlabel('Year')
plt.ylabel('Index (2014 = 100)')
plt.legend()
plt.grid(True)
plt.show()

2018 women suddenly increases a lot towards 2019. The lockdown in 2020 is visuably for both sexes. However, afterwards, women footballers continues to increase with a much higher rate than men.

Annual growth contributions

In [ ]:
# Step 1: Calculate year-over-year change for each category
idrakt_all['year_over_year_change'] = idrakt_all.groupby('sex')['value'].diff()

# Step 2: Calculate the contribution for each category
# First, create a pivot for the total values to align with the changes data
total_values_previous_year = idrakt_all[idrakt_all['sex'] == 'Sex, total'][['year', 'value']].rename(columns={'value': 'total_previous_year'})
total_values_previous_year['year'] += 1  # Aligning total of year n-1 with year n

# Merge this back into the main DataFrame
idrakt_all = pd.merge(idrakt_all, total_values_previous_year, on='year', how='left')

# Now calculate the contributions
idrakt_all['contribution'] = idrakt_all.apply(lambda x: x['year_over_year_change'] / x['total_previous_year'] if x['sex'] != 'Sex, total' else None, axis=1)

# Step 3: Pivot the data for plotting
pivot_contribution = idrakt_all.pivot(index='year', columns='sex', values='contribution')

# Visualize the contributions
pivot_contribution[['Men', 'Women']].plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Growth Contributions from Men and Women')
plt.xlabel('Year')
plt.ylabel('Growth p.a. (%)')
plt.legend(['Men', 'Women'])
plt.show()

The figure above shows that even though women increases a lot, the contributions to the overall growth is fairly dominated by men because of the larger base. 

# Analysis across geography

Plotting the number of football players by county

In [ ]:
# Filter the DataFrame for 'Sex, total'
idrakt_sex_total = idrakt[idrakt['sex'] == 'Sex, total']

# Create a series with the 2014 values
index_values = idrakt_sex_total[idrakt_sex_total['year'] == 2014].set_index('county')['value']

# Set the index
indexed_idrakt = idrakt_sex_total.set_index(['year', 'county'])

# Calculate the indexed values
indexed_idrakt['indexed_value'] = 100*indexed_idrakt['value'] / indexed_idrakt['value'].unstack().loc[2014]

# Resetting index for plotting
indexed_idrakt = indexed_idrakt.reset_index()

# Plot the indexed values
plt.figure(figsize=(10, 6))
for county, group in indexed_idrakt.groupby('county'): # Loop over the groups
    plt.plot(group['year'], group['indexed_value'], marker='o', label=county)

plt.title('Relative Development Indexed to 2014 Values')
plt.xlabel('Year')
plt.ylabel('Index (2014 = 100)')
plt.legend()
plt.grid(True)
plt.show()

Because it is difficult to see what is going on, we make a plot, splitting into the broader county definition, county_b

In [ ]:
# Filter the DataFrame for 'Sex, total'
idrakt_sex_total_2 = idrakt[idrakt['sex'] == 'Sex, total']

# Pivot the DataFrame
pivot_table = idrakt_sex_total_2.pivot_table(index='year', columns='county_b', values='value', aggfunc='sum')

# Calculate indexed values relative to 2014
indexed_values = pivot_table.div(pivot_table.loc[2014]) * 100

# Plot the indexed values
plt.figure(figsize=(10, 6))
for county_b in indexed_values.columns:
    plt.plot(indexed_values.index, indexed_values[county_b], marker='o', label=county_b)

plt.title('Relative Development Indexed to 2014 Values')
plt.xlabel('Year')
plt.ylabel('Index (2014 = 100)')
plt.legend()
plt.grid(True)
plt.show()


Making a plot of the sexes across counties

In [ ]:
from dataproject import calculate_geographic_shares

# Assuming df is your DataFrame with football player data
geographic_shares = calculate_geographic_shares(idrakt)

import matplotlib.pyplot as plt

# Creating a bar plot
geographic_shares.plot(kind='bar', x='county', y='share')
plt.title('Share of Football Players by Geography')
plt.xlabel('Geography')
plt.ylabel('Share')
plt.show()


# Conclusion

ADD CONCISE CONLUSION.

We see that...